### **Obtener noticias a través de Google**
_Por Juan Carlos Rodríguez-Raga y Andrés Mauricio Toloza Cruz_

Este código proporciona herramientas a través de Selenium y Newspaper3k para hacer web scrapping a noticias que aparezcan en la pestaña de noticias de Google. 

Este trabajo se hace en el marco del proyecto "What do they show?"

##### **Primer paso: descargar e importar nuestras librerias**
Las librerias que estaremos usando son Selenium y Newspaper3k  para descargarlas usamos el siguiente código:

    ! pip install selenium

    ! pip install newspaper3k

Tambien usaremos time y urllib, pero esas ya vienen en nuestra instalación de Python

In [ ]:
# Importamos las lbrerias necesarias

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

import newspaper
from newspaper import ArticleException

from urllib.parse import urlparse
import time
import json

##### **Segundo paso: extraer nuestras noticias de Google**

Para este paso es importante conocer nuestro buscador, en este caso Google. Para este ejemplo quiero descargar las noticias que hablen de protestas en los siguientes medios digitales de Colombia, esto de 2011 a 2024. 

La lista de esos medios y su link enlace es la siguiente:

| Nombre | Enlace |
| ------ | ------ |
| El Tiempo | eltiempo.com |
| El Espectador | elespectador.com |
| Semana | semana.com |
| El Colombiano | elcolombiano.com |
| El Pais Colombia | elpais.com.co |
| Caracol Radio | caracol.com.co |
| Portafolio | portafolio.com |
| Vanguardia | vanguardia.com |
| Minuto 30 | minuto30.com |
| Noticias RCN | noticiasrcn.com |
| Las dos Orillas | las2orillas.co |
| La Silla Vacia | lasillavacia.com |
| RCN Radio | rcn.com.co |
| W Radio | wradio.com.co |
| Dinero | dinero.com | 
| La Republica | larepublica.com |

Saber el enlace raiz es importante porque este nos va a permitir hacer busquedas en Google más especificas, para esto vamos a utilizar _[site:enlace raiz]_. En el caso del tiempo Google nos da el filtro _before:fecha_ y _after:fecha_. Hay que tener en cuenta que si Google cree que no encuentra resultados suficientes va a saltarse el filtro de tiempo.

Por ejemplo, si solo me interesará mirar protestas entre 2019 y 2022 podria hacer lo siguiente:

_protestas colombia after:2018-12-31 before:2023-01-01 [site:enlace raiz]_

In [ ]:
# Definimos un diccionario con nuestros enaleces raiz y un diccionario para los resultados

bases = [
        'eltiempo.com', 'elespectador.com', 'semana.com', 'elcolombiano.com',
         'elpais.com.co', 'caracol.com.co', 'portafolio.com', 'vanguardia.com',
         'minuto30.com', 'noticiasrcn.com', 'las2orillas.co', 'lasillavacia.com',
         'rcn.com.co', 'larepublica.com', 'dinero.com', 'wradio.com.co'
         ]

resultados = []

En este punto ya comenzamos a usar la libreria de Selenium, la cual nos permite ejecutar tareas automatizadas de webscrapping. En este caso, le doy de input el link de resultado de mi busqueda de Google y alteramos ciertos parametros para que puede alternar entre páginas de resultados.

In [ ]:
# Defino mi función scroll que permite hacer un scroll en la página web para cargar más noticias

def scroll():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

El siguiente ciclo es el que permite extraer los links de la noticia a través del XPATH del elemento HTML que contiene el link de esta.

In [ ]:
for base in bases:
    for pag in range(220, 301, 10):
        links = ['https://www.google.com/search?q=protestas+colombia+after:2019-12-31+before:2025-01-01+%5Bsite:'+str(base)+'%5D&sca_esv=a0ce93ff23945b86&rlz=1C5CHFA_enCO941CO941&tbm=nws&ei=f41UZuTZE4DJ1sQPhcSegAg&start='+str(pag)+'&sa=N&ved=2ahUKEwikldDg-q2GAxWApJUCHQWiB4AQ8tMDegQIAxAE&biw=1200&bih=846&dpr=1'
        ]
        for link in links:
            driver = webdriver.Chrome()
            driver.get(link)
            time.sleep(3)

            errores=0
            for j in range(2, 12):
                
                try:
                    scroll()
                    img_element = driver.find_element(By.XPATH, '//*[@id="rso"]/div/div/div['+str(j)+']/div/div/a')
                    # Agrego el link a la lista
                    resultados.append(img_element.get_attribute('href'))
                    print('Link added, iteration ' + str(j-1) + ' base: ' + str(base))
                except NoSuchElementException:
                    print('No image found for this iteration ('+ str(j) +'), moving to the next one.')
                    errores=errores+1
                    if errores == 2:
                        break
                    else:
                        continue
                
                time.sleep(5)
                driver.quit

In [ ]:
# Hago un checkpoint de los links obtenidos

import csv
# Abre (o crea) un archivo CSV en modo escritura ('w')
with open('links_checkpoint_news.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Escribe el título como la primera fila en el archivo CSV
    writer.writerow(['Links'])
    # Escribe cada enlace como una fila en el archivo CSV
    for link in resultados:
        # El método writerow espera una lista, así que ponemos el enlace en una lista
        writer.writerow([link])

##### **Tercer paso: extraer la información de los links y hacer pequeñas limpiezas**

En este último paso aplicaré pequeños filtros para limpiar mi lista de links y extraer la mayor cantidad de información con la libreria de Newspaper3k. Para finalmente filtrar noicias que mencionen palabras relacionadas con Protesta, Paro, Marcha, etc. 

In [ ]:
import pandas as pd

# Lee el archivo CSV
df_news = pd.read_csv('links_checkpoint_news.csv')

In [ ]:
# Elimina las filas duplicadas

df_news = df_news.drop_duplicates()

In [ ]:
# Pongo las filas de manera aleatoria (esto ayuda a tener mejores resultados con Newspaper3k)

df_news = df_news.sample(frac=1).reset_index(drop=True)

In [ ]:
# Define un diccionario para guardar los resultados

parse_data = {}

In [ ]:
# Realizo la extracción de noticias con Newspaper3k

import newspaper
import time
from urllib.parse import urlparse
from newspaper import ArticleException
import json

num = 1

for url in df_news.iloc['Links']:

    try: 

        article = newspaper.Article(url=url, language='es')
        article.download()
        article.parse()

        article_dict ={
            "title": str(article.title),
            "published_date": str(article.publish_date),
            "top_image": str(article.top_image),
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": str(article.text)
        }

        parse_data[article_dict["title"]] = article_dict

        print(str(num) + " - " + str(article_dict["title"]))

        time.sleep(5)

        num += 1

    except ArticleException:
        article_dict ={
            "title": "Error",
            "published_date": "Error",
            "top_image": "Error",
            "link": urlparse(url).netloc,
            "full_link": str(url),
            "text": "Error"
        }

        parse_data[article_dict["title"]] = article_dict

        print(str(num) + " - " + str(article_dict["title"]))

        time.sleep(5)

        num += 1

        continue

In [ ]:
# Convierto mis resultados en un DataFrame

df_extraidos = pd.DataFrame(list(parse_data.values()))

In [ ]:
# Exporto el dataframe a un archivo CSV
df_extraidos.to_csv('parse_data_chekpoint.csv', index=False)

In [ ]:
# Defino mi lista de palabras para el primer filtro

palabras = [
        'Protesta', 'Protestas', 'protesta', 'protestas',
        'Manifestaciones', 'Manifestación', 'manifestaciones', 'manifestación', 'Manifestacion', 'manifestacion',
        'Marcha', 'Marchas', 'marcha', 'marchas',
        'Paro', 'paro'
         ]

In [ ]:
# Aplicamos el filtro con las plabras

palabras_regex = '|'.join(palabras)  
df_filtro = df_extraidos[df_extraidos['text'].str.contains(palabras_regex, case=False, na=False)]

In [ ]:
# Guardamos el resultado en un archivo CSV

df_filtro.to_csv('parse_data_filtro1_chekpoint.csv', index=False)